# En fiktiv hälsostudie


In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

import src.metrics as M 
import src.viz as V 


In [2]:
plt.rcParams["figure.figsize"] = (7,5)
plt.rcParams["axes.grid"] = True
plt.rcParams["axes.grid.axis"] = "y"

In [3]:
#Importera data
df_start = pd.read_csv("data/health_study_dataset.csv")             

## Beskrivande analys

In [7]:
df = M.HelthAnalyser(df_start)

display(df.analys(df_start))

smoker = df.smoker(df_start)

sick = df.sick_df(df_start)

display(df.sick_sex(sick))

,age,weight,height,systolic_bp,cholesterol
mean,49.43,73.41,171.85,149.18,4.93
median,50.00,73.20,171.35,149.40,4.97
min,18.00,33.70,144.40,106.80,2.50
max,90.00,114.40,200.40,185.90,7.88


,sex,observations,mean_bp,mean_cholesterol,mean_age,weight_mean,smoker_count
0,F,18,151.250000,5.008333,49.888889,71.027778,8
1,M,29,150.468966,4.955517,51.172414,78.213793,11


In [ ]:
#Visualisering
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

V.histogram(axes[0], df["systolic_bp"], title = "Blodtryck", xlabel = "Blodtrycksvärde" )
V.boxplot(axes[1], [df.loc[df.sex == "F", "weight"], df.loc[df.sex == "M", "weight"]], ["F", "M"], "Män vs kvinnors vikt", "Kön", "Vikt")
V.bar(axes[2], smoker.smoker, smoker.observations, "Antal Rökare", "Rökare", "Antal")

plt.show()


### I ovan visuaiseringar kan vi från vår data utläsa följande:  
**Diagram 1:** Hur fördelningen av blodtryck ser ut.   
Det är flest i gruppen som har ett blodtryck på runt 150, och det finns en spridning från ca 110 till ca 180.   
**Diagram 2:** Hur fördelningen av vikt ser ut för kvinnor respektive män.   
Kvinnor har generellt en lägre vikt än män. Medelvikten för män ligger strax under 80kg medan medelvikten för kvinnor ligger under 70kg.  
**Diagram 3:** Hur många rökare respektive icke rökare det finns.  
Det är betydligt fler som inte röker än som röker.  

## Simulering
Hur många människor har sjukdomen. 

In [ ]:


# Simulering 

np.random.seed(42)
n = 1000
sim = np.random.choice([0.0, 1.0], p = [1-verklig_andel, verklig_andel], size = n)
simulerad_andel = sim.mean()


print(f"Andel personer som har sjukdomen: {verklig_andel:.2%}")
print(f"Simulerad andel personer som har sjukdomen: {simulerad_andel:.2%}")

In [ ]:
# Visualisering: 
jämf = pd.Series(
    [verklig_andel, simulerad_andel],
    index=["Verklig", "Simulerad"]
).reset_index()
V.spec_bar(jämf)

Simuleringen ger ett liknande resultat som vår verkliga andel från orginaldatan.

## Konfidensintervall för blodtryck

### Normal approximation

In [ ]:
#Tar fram ett stickprov och räknar ut konfidensintervall med hjälp av normalapproximation
np.random.seed(123)

n = 40 
stick= np.random.choice(df.systolic_bp, size= n)
mean_bp = df.systolic_bp.mean()

lo, hi, mean_stick, s, n, se = M.ci_mean_normal(stick)
print (f"Medelvärde finns inom blodtryck: {lo:.2f}, {hi:.2f} \n"
       f"Medelvärde för stickprov: {mean_stick:.2f} \n"
       f"Medelvärde från originaldata: {mean_bp:.2f} \n" 
       f"Standard avvikelse: {s:.2f}\n"
       f"Standardfel: {se:.2f}")    

In [ ]:
fig, ax = plt.subplots()
V.errorbar(ax, mean_stick, lo, hi, mean_bp)

## Bootstrap test 

In [ ]:
b_lo, b_hi, b_mean= M.ci_mean_bootstrap(stick)
print(f"""Mellan {b_lo:.2f} - {b_hi:.2f} kommer medelvärdet finnas 95% av gångerna, 
      Bootstrap medelvärde      : {b_mean:.2f}, 
      Medelvärde från stickprov : {float(mean_bp):.2f}""")

In [ ]:
fig, ax = plt.subplots(figsize= (7,3))
boot_info = np.array([np.mean(np.random.choice(stick, size=len(stick), replace= True)) for _ in range(1000)])
V.ax_boot(ax, boot_info, stick)

Diagrammet ovan visar vart medelvärdet för blodtryck hamnar vid 1000 olika stickprov med 40 personer per stickprov.

In [ ]:
print(f"Normal-CI: {lo:.2f}, {hi:.2f}")
print(f"Bootstrap-CI: {b_lo:.2f}, {b_hi:.2f}")
print("Stickprovsmedel:", mean_stick, "\nMedelvärde från original data:", mean_bp)

In [ ]:
# Testar hur ofta vårt orginal medelvärde fångas av de olika metoderna 
np.random.seed(123)
cov_normal = M.covers_true_mean(df.systolic_bp, mean_bp, "normal", n=40, trials=500)
cov_boot = M.covers_true_mean(df.systolic_bp, mean_bp, "bootstrap", n=40, trials=500)

In [ ]:
print(f"""Test för att se hur ofta de olika metoderna fångar vårt medelvärde:  
CI-normal   : {cov_normal:.4f}
CI-bootstrap: {cov_boot:.4f}""")

Konfidensintervallet för de två metoderna är i princip lika.  
Bottstrap-CI ger ett något större värdespann, men täcker ändå inte medelvärdet oftare än normalapproximation.  
Stickprovsmedlet ligger väldigt nära medelvärdet från orginaldata och kan därför ses som representativt. 

# Hypotesprövning
### Hypotes: Rökare har högre medel-blodtryck än icke-rökare.

In [ ]:
display(smoker)

Metodval: Welch t-test med alternativ greater: Detta då vi jämför två olika stora grupper och antar att rökare har högre blodtryck än den andra. Källa: Joakim Lindh och https://en.wikipedia.org/wiki/Welch%27s_t-test

In [ ]:
from scipy import stats

non_smoker = df.loc[df.smoker == "No", "systolic_bp"].values
is_smoker = df.loc[df.smoker == "Yes", "systolic_bp"].values

t_stat_w, p_val_w = stats.ttest_ind(is_smoker, non_smoker, equal_var=False, alternative = "greater")

print(f"Welch t-test: t = {t_stat_w:.3f}, p = {p_val_w:.4f}")

Positivt t-värde visar att rökare har ett högre medel blodtryck än icke rökare. Dock är resultatet väldigt nära noll och p-värdet är långt över signifikantnivå på 0,05 vilket gör att vi inte kan utesluta att det inte finns någon skillnad mellan gruppernas blodtryck.

In [ ]:
def power_by_simulation(n_sim, mean_non_smoker, mean_is_smoker, n_non_smoker, n_is_smoker, std, n = 1, alpha= 0.05):
    np.random.seed(7)
    count_significant = 0 
    for _ in range(n_sim):
        non_smoker = np.random.normal(mean_non_smoker, std, (n_non_smoker * n))
        is_smoker = np.random.normal(mean_is_smoker, std, (n_is_smoker *n))
        _, p_val = stats.ttest_ind(is_smoker, non_smoker, equal_var=False, alternative="greater")
        if p_val < alpha: 
            count_significant += 1
    return count_significant / n_sim

In [ ]:
n_sim = 5000
alpha = 0.05 
n_non_smoker = smoker["observations"][0]
n_is_smoker = smoker["observations"][1]
mean_non_smoker = smoker["proportions"][0]
mean_is_smoker = smoker["proportions"][1]
std = df["systolic_bp"].std()
n = 1

pwr = power_by_simulation(n_sim, mean_non_smoker, mean_is_smoker, n_non_smoker, n_is_smoker, std, n, alpha)
print(f"Power för observerad skillnad: {pwr}")

Med en power på 11,36% kan vi inte dra några säkra slutsatser på att det finns en verklig skillnad mellan gruppernas blodtryck. Ett större deltagar antal behövs för att få ett mer stabilt resultat.

In [ ]:
samples_try = [1, 10, 20, 25, 28, 30]
sim_pwr = []
for n in samples_try:
    pwr = power_by_simulation(n_sim, mean_non_smoker, mean_is_smoker, n_non_smoker, n_is_smoker, std, n, alpha)
    sim_pwr.append(pwr)

pd.DataFrame({f"n_non_smoker": n_non_smoker, "n_is_smoker": n_is_smoker, "groupsize_times_n": samples_try, "simulated_power": sim_pwr})

In [ ]:
fig, ax = plt.subplots()
V.plot(ax, samples_try, sim_pwr)
plt.show()

In [ ]:
pwr_28 = power_by_simulation(n_sim, mean_non_smoker, mean_is_smoker, n_non_smoker, n_is_smoker, std, 28, alpha)
pwr_28

In [ ]:
need_smokers = n_is_smoker * 28
need_non_smoker = n_non_smoker * 28

need_non_smoker, need_smokers, (need_smokers + need_non_smoker)

För att få ett tillräkligt säkert resultat skulle grupperna behöva vara 28 gånger större.   
Det innebär att man skulle behöva ungefär 22 400 deltagare för att få ett tillförlitligt resultat   
och se om det på riktigt finns någon skillnad i blodtryck mellan rökare och icke rökare. 